ChatGPT helped me understand more about what this archetecture was and how to build this

In [4]:
pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.7 MB/s eta 0:00:00


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Dropout, MultiHeadAttention, LayerNormalization, Add, Lambda
from tensorflow.keras.models import Model

from keras import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

tf.get_logger().setLevel('INFO')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Read in the data
def read_file(filename):
    directory = '/content/drive/My Drive/learning_data/'
    location = directory + filename+'.csv'
    df = pd.read_csv(location, index_col=False)
    return df


In [7]:
## Read in all data files
X_test = read_file('X_test_yields')
X_val = read_file('X_val_yields')
Y_test = read_file('Y_test_yields')
Y_train = read_file('Y_train_yields')
Y_val = read_file('Y_val_yields')
X_train = read_file('X_train_yields')

In [8]:
X_train[['state_ansi','state']].value_counts()

,,count
state_ansi,state,
19,iowa,2134
17,illinois,2106
18,indiana,1856
31,nebraska,1824
21,kentucky,1821
39,ohio,1765
29,missouri,1729
27,minnesota,1607
37,north carolina,1586


In [9]:
## print shapes
print('\nX_train:',X_train.shape,
      '\nY_train:',Y_train.shape,
      '\nX_val:',X_val.shape,
      '\nY_val:',Y_val.shape,
      '\nX_test:',X_test.shape,
      '\nY_test:',Y_test.shape
      )


X_train: (17727, 1000) 
Y_train: (17727, 1) 
X_val: (1479, 1000) 
Y_val: (1479, 1) 
X_test: (603, 1000) 
Y_test: (603, 1)


In [10]:
X_colz = X_train.columns.tolist()
X_colz

['year',
 'state',
 'state_ansi',
 'county',
 'county_ansi',
 'ag_district',
 'ag_district_code',
 'irrig_yield',
 'non_irrig_yield',
 'all_ge_var',
 'ht_only',
 'bt_only',
 'stacked_genes',
 'jan_rain',
 'feb_rain',
 'mar_rain',
 'apr_rain',
 'may_rain',
 'jun_rain',
 'jul_rain',
 'aug_rain',
 'sep_rain',
 'oct_rain',
 'nov_rain',
 'dec_rain',
 'jan_temp',
 'feb_temp',
 'mar_temp',
 'apr_temp',
 'may_temp',
 'jun_temp',
 'jul_temp',
 'aug_temp',
 'sep_temp',
 'oct_temp',
 'nov_temp',
 'dec_temp',
 'state_county',
 'loc_georgia_appling',
 'loc_georgia_atkinson',
 'loc_georgia_bacon',
 'loc_georgia_baker',
 'loc_georgia_banks',
 'loc_georgia_bartow',
 'loc_georgia_ben hill',
 'loc_georgia_berrien',
 'loc_georgia_bleckley',
 'loc_georgia_brantley',
 'loc_georgia_brooks',
 'loc_georgia_bryan',
 'loc_georgia_bulloch',
 'loc_georgia_burke',
 'loc_georgia_calhoun',
 'loc_georgia_candler',
 'loc_georgia_carroll',
 'loc_georgia_catoosa',
 'loc_georgia_chattooga',
 'loc_georgia_clay',
 'loc_geo

In [11]:
X_train_clean = X_train[[
 'year',
 'state_ansi',
 'county_ansi',
 'ag_district_code',
 'state_county',
 'all_ge_var',
 'ht_only',
 'bt_only',
 'stacked_genes',
 'jan_rain',
 'feb_rain',
 'mar_rain',
 'apr_rain',
 'may_rain',
 'jun_rain',
 'jul_rain',
 'aug_rain',
 'sep_rain',
 'oct_rain',
 'nov_rain',
 'dec_rain',
 'jan_temp',
 'feb_temp',
 'mar_temp',
 'apr_temp',
 'may_temp',
 'jun_temp',
 'jul_temp',
 'aug_temp',
 'sep_temp',
 'oct_temp',
 'nov_temp',
 'dec_temp'
]]

Creating defs for making reproduceable datasets to run many models to find the best predictions

In [12]:
## Create a list of columns that we are keeping

cols=[
 'year',
 'state_ansi',
 'county_ansi',
 'ag_district_code',
 'state_county',
 'all_ge_var',
 'ht_only',
 'bt_only',
 'stacked_genes',
 'jan_rain',
 'feb_rain',
 'mar_rain',
 'apr_rain',
 'may_rain',
 'jun_rain',
 'jul_rain',
 'aug_rain',
 'sep_rain',
 'oct_rain',
 'nov_rain',
 'dec_rain',
 'jan_temp',
 'feb_temp',
 'mar_temp',
 'apr_temp',
 'may_temp',
 'jun_temp',
 'jul_temp',
 'aug_temp',
 'sep_temp',
 'oct_temp',
 'nov_temp',
 'dec_temp'
]

In [13]:
def col_keep(col_list,X):
  """takes a list of columns and the X df and returns only the wanted columns"""
  dropped_col = X[col_list]
  return dropped_col

In [14]:
## Create a function that removes the NAs from the below function

def remove_na(X,Y):
  """takes X and Y for either training / validation / test sets and removes na rows for both"""
  na_idx = X.isna().any(axis=1)
  Y_clean = Y[~na_idx]
  X_clean = X[~na_idx]

  return X_clean, Y_clean


This model will look at 2 things, rain and temperature data in relationship to time AND then also the static number inputs

In [15]:
### create X train,test,val datasets with only the wanted columns
## Keeping only the ge data

X_train_cols = col_keep(cols,X_train)
X_train_cols.shape
X_val_cols = col_keep(cols,X_val)
X_val_cols.shape
X_test_cols = col_keep(cols,X_test)
X_test_cols.shape

print('X_train_cols', X_train_cols.shape,
      '\nX_val_cols:',X_val_cols.shape,
      '\nX_test_cols:',X_test_cols.shape
      )

X_train_cols (17727, 33) 
X_val_cols: (1479, 33) 
X_test_cols: (603, 33)


In [16]:
## Removing the nulls from datasets:

## Train
X_train_cols_clean, Y_train_cols_clean = remove_na(X_train_cols,Y_train)
print('X_train_cols_clean:', X_train_cols_clean.shape, '\nY_train_cols_clean:', Y_train_cols_clean.shape)

X_val_cols_clean, Y_val_cols_clean = remove_na(X_val_cols,Y_val)
print('X_val_cols_clean:', X_val_cols_clean.shape, '\nY_val_cols_clean:', Y_val_cols_clean.shape)

X_test_cols_clean, Y_test_cols_clean= remove_na(X_test_cols,Y_test)
print('X_test_cols_clean:', X_test_cols_clean.shape, '\nY_test_cols_clean:', Y_test_cols_clean.shape)




X_train_cols_clean: (12418, 33) 
Y_train_cols_clean: (12418, 1)
X_val_cols_clean: (1092, 33) 
Y_val_cols_clean: (1092, 1)
X_test_cols_clean: (446, 33) 
Y_test_cols_clean: (446, 1)


In [17]:
clean = X_train_cols_clean['state_ansi'].value_counts()
og = X_train[['state_ansi','state']].value_counts()

og

,,count
state_ansi,state,
19,iowa,2134
17,illinois,2106
18,indiana,1856
31,nebraska,1824
21,kentucky,1821
39,ohio,1765
29,missouri,1729
27,minnesota,1607
37,north carolina,1586


In [18]:
### Transform Dataset into 3D Tensor for Transformer or RNN model e.g. (num_samples, sequence_length, num_features)
### here we are building a sequence input tensor for a multivariate time series

In [19]:
## Pull the data out into 2 numpy b/c TF doesn't work with Data Frames (ChatGPT wrote alot of this code)


def prepare_data(df,
                 dfy,
                 static_cols1,
                 scaler_seq=None,
                 scaler_static=None,
                 y_scaler=None,
                 fit=False):
    """
    Prepares static features, sequence features (rain/temp), and target y from a DataFrame.

    Params:
        df: pandas DataFrame
        dfy : pandas DF with y vals,
        static_cols1: list of columns that are static, but not locations
        scaler_seq: StandardScaler instance for sequence features
        scaler_static: Scaler (e.g. MinMaxScaler) for static features
        y_scaler: Scaler for target variable (optional)
        fit: if True, fits the scalers on this data; else just transforms

    Returns:
        X_static_scaled: np.array of shape (samples, num_static_features)
        X_seq_scaled: np.array of shape (samples, 7, 2)
        y_scaled: np.array of shape (samples,) or (samples, 1)
        scalers: dict of fitted scalers for re-use on val/test
    """
    import numpy as np
    from sklearn.preprocessing import StandardScaler, MinMaxScaler

    months = ['jan','feb','mar','apr','may','jun','jul'] #,'aug','sep','oct','nov','dec']
    rain_cols = [f"{m}_rain" for m in months]
    temp_cols = [f"{m}_temp" for m in months]

    # --- Sequence input (rain + temp) ---
    rain_data = df[rain_cols].to_numpy()
    temp_data = df[temp_cols].to_numpy()
    seq_raw = np.stack([rain_data, temp_data], axis=-1)  # (samples, 12, 2)

    # Flatten for scaling
    seq_flat = seq_raw.reshape(-1, 2)
    if fit:
        scaler_seq = StandardScaler()
        seq_flat_scaled = scaler_seq.fit_transform(seq_flat)
    else:
        seq_flat_scaled = scaler_seq.transform(seq_flat)

    X_seq_scaled = seq_flat_scaled.reshape(seq_raw.shape)

    # --- Static input ---
    static_cols = static_cols1 # Add more if you want (like 'state_ansi', etc.)
    X_static_raw = df[static_cols].to_numpy()

    if fit:
        scaler_static = MinMaxScaler()
        X_static_scaled = scaler_static.fit_transform(X_static_raw)
    else:
        X_static_scaled = scaler_static.transform(X_static_raw)

    # --- Target variable (if available) ---
    y_scaled = None
    if 'yield_bu_acre' in dfy.columns:
        y_raw = dfy['yield_bu_acre'].to_numpy().reshape(-1, 1)
        if fit:
            y_scaler = StandardScaler()
            y_scaled = y_scaler.fit_transform(y_raw)
        else:
            y_scaled = y_scaler.transform(y_raw)

    return X_static_scaled, X_seq_scaled, y_scaled, {
        'scaler_seq': scaler_seq,
        'scaler_static': scaler_static,
        'y_scaler': y_scaler
    }


In [20]:
# First on training data (fit=True)


static_cols1 = ['year', 'all_ge_var', 'stacked_genes']

# --- Train (fit=True: learn scaling on training data) ---
X_static_train, X_seq_train, y_train, scalers = prepare_data(
    df=X_train_cols_clean,
    dfy=Y_train_cols_clean,
    static_cols1=static_cols1,
    fit=True
)

# --- Validation (reuse scalers from training) ---
X_static_val, X_seq_val, y_val, _ = prepare_data(
    df=X_val_cols_clean,
    dfy=Y_val_cols_clean,
    static_cols1=static_cols1,
    scaler_seq=scalers['scaler_seq'],
    scaler_static=scalers['scaler_static'],
    y_scaler=scalers['y_scaler'],
    fit=False
)

# --- Test (also reuse scalers) ---
X_static_test, X_seq_test, y_test, _ = prepare_data(
    df=X_test_cols_clean,
    dfy=Y_test_cols_clean,
    static_cols1=static_cols1,
    scaler_seq=scalers['scaler_seq'],
    scaler_static=scalers['scaler_static'],
    y_scaler=scalers['y_scaler'],
    fit=False
)

In [21]:
def check(name, X_static, X_seq, y):
    print(f"\n--- {name.upper()} ---")
    print(f"X_static: {X_static.shape}")
    print(f"X_seq:    {X_seq.shape}")
    print(f"y:        {y.shape}")
    print(f"Static range:     {X_static.min():.2f} to {X_static.max():.2f}")
    print(f"Seq rain/temp:    {X_seq.min():.2f} to {X_seq.max():.2f}")
    print(f"Target (scaled):  {y.min():.2f} to {y.max():.2f}")

check("train", X_static_train, X_seq_train, y_train)
check("val", X_static_val, X_seq_val, y_val)
check("test", X_static_test, X_seq_test, y_test)


--- TRAIN ---
X_static: (12418, 3)
X_seq:    (12418, 7, 2)
y:        (12418, 1)
Static range:     0.00 to 1.00
Seq rain/temp:    -2.74 to 6.65
Target (scaled):  -4.22 to 2.76

--- VAL ---
X_static: (1092, 3)
X_seq:    (1092, 7, 2)
y:        (1092, 1)
Static range:     0.83 to 1.10
Seq rain/temp:    -2.63 to 4.01
Target (scaled):  -3.09 to 2.57

--- TEST ---
X_static: (446, 3)
X_seq:    (446, 7, 2)
y:        (446, 1)
Static range:     0.89 to 1.14
Seq rain/temp:    -1.97 to 5.18
Target (scaled):  -2.15 to 2.81


In [22]:
## encode state_county -> using ordinal encoder so it will handle unknownv county-state values in the dataset
def encode_state_county_fit(df):
  ord_state_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
  df=df.copy()
  df.loc[:,'state_county_id'] = ord_state_encoder.fit_transform(df[['state_county']])
  return df, ord_state_encoder

def encode_state_county_transform(df, encoder):
    df = df.copy()
    df['state_county_id'] = encoder.transform(df[['state_county']])
    return df

In [23]:
# Fit on train
X_train_cols_clean, state_encoder = encode_state_county_fit(X_train_cols_clean)

# Transform on val and test
X_val_cols_clean = encode_state_county_transform(X_val_cols_clean, state_encoder)
X_test_cols_clean = encode_state_county_transform(X_test_cols_clean, state_encoder)

In [24]:
## embed location columns
embed_cols = ['state_ansi','ag_district_code','state_county_id']
input_dims = {col: X_train_cols_clean[col].nunique() for col in embed_cols}
print(input_dims)

{'state_ansi': 7, 'ag_district_code': 9, 'state_county_id': 657}


In [25]:
## using the mean Y as the baseline model

baseline_model = Y_train_cols_clean.mean()
Y_train_std = Y_train_cols_clean.std()
print('Baseline Mode:',baseline_model,
      '\n Baseline with std:', baseline_model+Y_train_std)

Baseline Mode: yield_bu_acre    156.719931
dtype: float64 
 Baseline with std: yield_bu_acre    189.318308
dtype: float64


In [26]:

def evaluate_and_report(model,
                        X_static_train, X_seq_train, X_train_df, y_train_scaled,
                        X_static_val, X_seq_val, X_val_df, y_val_scaled,
                        y_scaler):

    # --- Prepare input function ---
    def get_inputs(X_static, X_seq, X_df):
        return {
            "numeric_input": X_static,
            "state_ansi": X_df["state_ansi"].astype(np.int32).to_numpy(),
            "state_county_id": X_df["state_county_id"].astype(np.int32).to_numpy(),
            "ag_district_code": X_df["ag_district_code"].astype(np.int32).to_numpy(),
            "seq_input": X_seq
        }

    # --- Predict and inverse transform ---
    train_inputs = get_inputs(X_static_train, X_seq_train, X_train_df)
    val_inputs = get_inputs(X_static_val, X_seq_val, X_val_df)

    y_train_pred_scaled = model.predict(train_inputs)
    y_val_pred_scaled = model.predict(val_inputs)

    y_train_true = y_scaler.inverse_transform(y_train_scaled)
    y_val_true = y_scaler.inverse_transform(y_val_scaled)
    y_train_pred = y_scaler.inverse_transform(y_train_pred_scaled)
    y_val_pred = y_scaler.inverse_transform(y_val_pred_scaled)

    # --- Compute metrics ---
    train_mae = mean_absolute_error(y_train_true, y_train_pred)
    val_mae = mean_absolute_error(y_val_true, y_val_pred)

    train_rmse = np.sqrt(mean_squared_error(y_train_true, y_train_pred))
    val_rmse = np.sqrt(mean_squared_error(y_val_true, y_val_pred))

    train_mean = np.mean(y_train_true)
    norm_train_rmse = train_rmse / train_mean
    norm_val_rmse = val_rmse / train_mean
    norm_train_mae = train_mae / train_mean
    norm_val_mae = val_mae / train_mean

    # --- Generalization ratio ---
    generalization_ratio = norm_val_mae / norm_train_mae

    # --- Print report ---
    print(f"Generalization Ratio (normalized MAE): {generalization_ratio:.2f}x\n")

    print(f"Final Training RMSE (normalized): {norm_train_rmse:.4f}")
    print(f"Final Validation RMSE (normalized): {norm_val_rmse:.4f}\n")

    print(f"Final Training RMSE (original): {train_rmse:.2f} bu/acre")
    print(f"Final Validation RMSE (original): {val_rmse:.2f} bu/acre")

    # return {
    #     "train_rmse": train_rmse,
    #     "val_rmse": val_rmse,
    #     "norm_train_rmse": norm_train_rmse,
    #     "norm_val_rmse": norm_val_rmse,
    #     "train_mae": train_mae,
    #     "val_mae": val_mae,
    #     "norm_train_mae": norm_train_mae,
    #     "norm_val_mae": norm_val_mae,
    #     "generalization_ratio": generalization_ratio
    # }


In [27]:
def build_model(X,X_val,num_cols, num_input, embedding_dim, transformer_dim=64, num_heads=6):
  """ This model takes X and X_val inputs and embeds the locations, calculates the input_dim for each column
  modifies the numeric input to include only values specificed in the num_cols field
  """
  tf.keras.backend.clear_session()
  tf.random.set_seed(0)

  ## fixing naming stuff

  state_input = Input(shape=(1,), name='state_ansi')
  county_input = Input(shape=(1,), name='state_county_id')
  district_input = Input(shape=(1,), name='ag_district_code')
  numeric_input = Input(shape=(num_input.shape[1],), name='numeric_input')
  seq_input = Input(shape=(7, 2), name="seq_input")

  # ---- All numeric inputs here ---- #
  # all numeric inputs here

  num_input = num_input[num_cols]
  numeric_input = Input(shape=(num_input.shape[1],), name='numeric_input')
  dense_static = Dense(64, activation="relu")(numeric_input)

  # --- embed  all location data here --- #
  ## Calc inputs for locations
  state_input_dim = int(max(X['state_ansi'].max(), X_val['state_ansi'].max()) + 1)
  county_input_dim = int(max(X['state_county_id'].max(), X_val['state_county_id'].max()) + 1)
  dist_input_dim = int(max(X['ag_district_code'].max(), X_val['ag_district_code'].max()) + 1)

  ## add embededded inputs here

  state_input = Input(shape=(1,), name='state_ansi')
  county_input = Input(shape=(1,), name='state_county_id')
  district_input = Input(shape=(1,), name='ag_district_code')

  #Embed here
  state_embed = Embedding(input_dim=state_input_dim, output_dim=3)(state_input)
  district_embed = Embedding(input_dim=dist_input_dim, output_dim=30)(district_input)
  county_embed = Embedding(input_dim=county_input_dim, output_dim=4)(county_input)

  # Flatten location here
  county_embed = Flatten()(county_embed)
  state_embed = Flatten()(state_embed)
  district_embed = Flatten()(district_embed)

  # --- Sequential Input (rain + temp over 12 months) ---
  seq_input = Input(shape=(7, 2), name="seq_input")  # shape: (batch, 4, 2)

     # --- Positional Embedding ---
  pos_embedding_layer = Embedding(input_dim=7, output_dim=2, name="pos_embedding")
  month_embed_weights = pos_embedding_layer(tf.range(7))  # shape: (4, 2)

  def add_positional_encoding(x):
      batch_size = tf.shape(x)[0]
      month_embed = tf.broadcast_to(month_embed_weights, [batch_size, 7, 2])
      return x + month_embed

  seq_with_pos = Lambda(add_positional_encoding,
                      name="add_pos_encoding",
                      output_shape=lambda s: s)(seq_input)


  # --- Transformer Attention Block ---
  ## the purpose of the below architectural is to use transformers to identify and learn patterns
  ## with rain and temperature.

  ## First we expand the sequence using a dense layer to 32 to give
  ## the model room

  seq_expanded = Dense(32)(seq_with_pos)  # (7, 2) → (7, 32)

  ## Then we add a multiheadattention block with 6 heads (to learn 6 patterns)
  ## with 64 for each head to allow for deeper learning. We also are using the
  ## sequence with position encodings for the query and key/value to compare to
  ## itself and learn time related patterns (e.g. how jan -> feb)

  # attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=transformer_dim)(seq_expanded, seq_expanded)
  # attn_output = Dropout(0.1)(attn_output)
  # attn_output = Add()([seq_expanded, attn_output])  # Residual
  # attn_output = LayerNormalization()(attn_output)


  # First attention block
  attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=transformer_dim)(seq_expanded, seq_expanded)
  attn_output = Dropout(0.15)(attn_output)  # More dropout
  attn_output = Add()([seq_expanded, attn_output])
  attn_output = LayerNormalization()(attn_output)

  # Second attention block
  attn_output2 = MultiHeadAttention(num_heads=num_heads, key_dim=transformer_dim)(attn_output, attn_output)
  attn_output2 = Dropout(0.25)(attn_output2)  # More dropout
  attn_output2 = Add()([attn_output, attn_output2])
  attn_output2 = LayerNormalization()(attn_output2)


  # --- Feedforward Layer (with Residual) ---
  ffn = Dense(128)(attn_output2)  # (7, 32) → (7, 128)
  ffn = Dense(32)(ffn)  # (7, 128) → (7, 32)
  ffn = Add()([attn_output, ffn])
  x_seq = LayerNormalization()(ffn)
  x_seq = Flatten()(x_seq)


  #Concatenate all data together
  ## removed: state_embed, district_embed,
  x = Concatenate()([county_embed, dense_static, x_seq])


  # Begin with hidden layers
  x = Dense(units = 1024, activation='relu')(x)
  x = Dropout(rate = 0.5)(x)
  x = Dense(units = 512, activation='relu')(x)
  x = Dropout(rate = 0.2)(x)
  x = Dense(units = 64, activation='relu')(x)
  x = Dropout(rate = 0.3)(x)

  # Output
  Yield = Dense(units=1, activation='linear', name='Yield')(x)

  ## build_model
  ## removed: state_embed, district_embed,
  model = tf.keras.Model(
      inputs=[ county_input, numeric_input, seq_input], ## may or may not need to have seq_with_pos
      outputs=Yield,
      name='Yield_1_fx'
  )
  ## model_compile

  model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=0.003),
              loss="mse",
              metrics=[
    tf.keras.metrics.MeanAbsoluteError(name='mae'),
    tf.keras.metrics.RootMeanSquaredError(name='rmse')
    ])

  return model

In [28]:
def model_fit(model,
              X_static_train,  # gene, year
              X_seq_train,     # rain/temp
              X_train_df,      # location cols

              X_static_val,
              X_seq_val,
              X_val_df,

              y_train,
              y_val,

              epochs=30,
              batch_size=32):

    import numpy as np
    import tensorflow as tf

    # --- Ensure correct data types for embeddings ---
    embedding_cols = ["state_ansi", "state_county_id", "ag_district_code"]
    for col in embedding_cols:
        X_train_df[col] = X_train_df[col].astype(np.int32)
        X_val_df[col] = X_val_df[col].astype(np.int32)

    # --- Package training inputs ---
    train_inputs = {
        "numeric_input": X_static_train,
        "state_ansi": X_train_df["state_ansi"].to_numpy(),
        "state_county_id": X_train_df["state_county_id"].to_numpy(),
        "ag_district_code": X_train_df["ag_district_code"].to_numpy(),
        "seq_input": X_seq_train
    }

    # --- Package validation inputs ---
    val_inputs = {
        "numeric_input": X_static_val,
        "state_ansi": X_val_df["state_ansi"].to_numpy(),
        "state_county_id": X_val_df["state_county_id"].to_numpy(),
        "ag_district_code": X_val_df["ag_district_code"].to_numpy(),
        "seq_input": X_seq_val
    }

    # --- Fit the model ---
    history = model.fit(
        x=train_inputs,
        y=y_train,
        validation_data=(val_inputs, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
        ],
        verbose=1
    )

    return history, history.history


In [29]:

model = build_model(
    X=X_train_cols_clean,
    X_val=X_val_cols_clean,
    num_cols=static_cols1,
    num_input=pd.DataFrame(X_static_train, columns=static_cols1),
    embedding_dim=4,
    transformer_dim=64,
    num_heads=4
)

history, history_dict = model_fit(
    model,
    X_static_train=X_static_train,
    X_seq_train=X_seq_train,
    X_train_df=X_train_cols_clean,

    X_static_val=X_static_val,
    X_seq_val=X_seq_val,
    X_val_df=X_val_cols_clean,

    y_train=y_train,
    y_val=y_val,

    epochs=30,
    batch_size=32
)


Epoch 1/30
389/389 ━━━━━━━━━━━━━━━━━━━━ 29s 43ms/step - loss: 1.0670 - mae: 0.7738 - rmse: 1.0146 - val_loss: 0.4762 - val_mae: 0.5433 - val_rmse: 0.6901
Epoch 2/30
389/389 ━━━━━━━━━━━━━━━━━━━━ 16s 41ms/step - loss: 0.4539 - mae: 0.5161 - rmse: 0.6731 - val_loss: 0.3917 - val_mae: 0.4712 - val_rmse: 0.6259
Epoch 3/30
389/389 ━━━━━━━━━━━━━━━━━━━━ 21s 41ms/step - loss: 0.3862 - mae: 0.4736 - rmse: 0.6210 - val_loss: 0.3468 - val_mae: 0.4577 - val_rmse: 0.5889
Epoch 4/30
389/389 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - loss: 0.3567 - mae: 0.4562 - rmse: 0.5971 - val_loss: 0.2972 - val_mae: 0.4169 - val_rmse: 0.5452
Epoch 5/30
389/389 ━━━━━━━━━━━━━━━━━━━━ 17s 44ms/step - loss: 0.3390 - mae: 0.4406 - rmse: 0.5820 - val_loss: 0.3043 - val_mae: 0.4159 - val_rmse: 0.5517
Epoch 6/30
389/389 ━━━━━━━━━━━━━━━━━━━━ 19s 40ms/step - loss: 0.3077 - mae: 0.4239 - rmse: 0.5547 - val_loss: 0.3201 - val_mae: 0.4204 - val_rmse: 0.5658
Epoch 7/30
389/389 ━━━━━━━━━━━━━━━━━━━━ 16s 41ms/step - loss: 0.3046 - mae: 

In [30]:
evaluate_and_report(
    model,
    X_static_train, X_seq_train, X_train_cols_clean, y_train,
    X_static_val, X_seq_val, X_val_cols_clean, y_val,
    scalers['y_scaler']
)


389/389 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Generalization Ratio (normalized MAE): 1.18x

Final Training RMSE (normalized): 0.0966
Final Validation RMSE (normalized): 0.1134

Final Training RMSE (original): 15.14 bu/acre
Final Validation RMSE (original): 17.77 bu/acre


In [31]:
y_baseline = np.full_like(Y_val_cols_clean, fill_value=np.mean(Y_train_cols_clean))

y_baseline.shape


(1092, 1)

In [32]:

mse_baseline = mean_squared_error(Y_val_cols_clean, y_baseline)
RMSE_baseline = np.sqrt(mse_baseline)
baseline_mae = mean_absolute_error(Y_val_cols_clean, y_baseline)


print(f"Baseline MAE: {baseline_mae:.4f}")
print(f"Baseline RMSE: {RMSE_baseline:.4f}")


Baseline MAE: 34.4995
Baseline RMSE: 39.5707


In [33]:
test_inputs = {
    "numeric_input": X_static_test,  # Your static features for test
    "state_ansi": X_test_cols_clean["state_ansi"].to_numpy(),
    "state_county_id": X_test_cols_clean["state_county_id"].to_numpy(),
    "ag_district_code": X_test_cols_clean["ag_district_code"].to_numpy(),
    "seq_input": X_seq_test  # Your sequential weather data for test
}

# Run test predictions (scaled)
test_predictions_scaled = model.predict(test_inputs)

# Unscale back to original bushels per acre
test_predictions_original = scalers['y_scaler'].inverse_transform(test_predictions_scaled)
y_test_original = scalers['y_scaler'].inverse_transform(y_test)

# Calculate metrics in original units
test_rmse_original = np.sqrt(mean_squared_error(y_test_original, test_predictions_original))
test_mae_original = mean_absolute_error(y_test_original, test_predictions_original)

print(f"Test RMSE: {test_rmse_original:.2f} bu/acre")
print(f"Test MAE: {test_mae_original:.2f} bu/acre")

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step
Test RMSE: 16.60 bu/acre
Test MAE: 13.49 bu/acre


In [34]:
def evaluate_and_report2(model,
                        X_static_train, X_seq_train, X_train_df, y_train_scaled,
                        X_static_val, X_seq_val, X_val_df, y_val_scaled,
                        X_static_test, X_seq_test, X_test_df, y_test_scaled,
                        y_scaler):

    # --- Prepare input function ---
    def get_inputs(X_static, X_seq, X_df):
        return {
            "numeric_input": X_static,
            "state_ansi": X_df["state_ansi"].astype(np.int32).to_numpy(),
            "state_county_id": X_df["state_county_id"].astype(np.int32).to_numpy(),
            "ag_district_code": X_df["ag_district_code"].astype(np.int32).to_numpy(),
            "seq_input": X_seq
        }

    # --- Predict and inverse transform ---
    train_inputs = get_inputs(X_static_train, X_seq_train, X_train_df)
    val_inputs = get_inputs(X_static_val, X_seq_val, X_val_df)
    test_inputs = get_inputs(X_static_test, X_seq_test, X_test_df)

    y_train_pred_scaled = model.predict(train_inputs)
    y_val_pred_scaled = model.predict(val_inputs)
    y_test_pred_scaled = model.predict(test_inputs)

    y_train_true = y_scaler.inverse_transform(y_train_scaled)
    y_val_true = y_scaler.inverse_transform(y_val_scaled)
    y_test_true = y_scaler.inverse_transform(y_test_scaled)

    y_train_pred = y_scaler.inverse_transform(y_train_pred_scaled)
    y_val_pred = y_scaler.inverse_transform(y_val_pred_scaled)
    y_test_pred = y_scaler.inverse_transform(y_test_pred_scaled)

    # --- Compute metrics ---
    # MAE
    train_mae = mean_absolute_error(y_train_true, y_train_pred)
    val_mae = mean_absolute_error(y_val_true, y_val_pred)
    test_mae = mean_absolute_error(y_test_true, y_test_pred)

    # MSE
    train_mse = mean_squared_error(y_train_true, y_train_pred)
    val_mse = mean_squared_error(y_val_true, y_val_pred)
    test_mse = mean_squared_error(y_test_true, y_test_pred)

    # RMSE
    train_rmse = np.sqrt(train_mse)
    val_rmse = np.sqrt(val_mse)
    test_rmse = np.sqrt(test_mse)

    # Normalized metrics (using training mean)
    train_mean = np.mean(y_train_true)

    norm_train_mae = train_mae / train_mean
    norm_val_mae = val_mae / train_mean
    norm_test_mae = test_mae / train_mean

    norm_train_mse = train_mse / (train_mean ** 2)
    norm_val_mse = val_mse / (train_mean ** 2)
    norm_test_mse = test_mse / (train_mean ** 2)

    norm_train_rmse = train_rmse / train_mean
    norm_val_rmse = val_rmse / train_mean
    norm_test_rmse = test_rmse / train_mean

    # --- Generalization ratios ---
    train_val_ratio = norm_val_mae / norm_train_mae
    val_test_ratio = norm_test_mae / norm_val_mae

    # --- Print comprehensive report ---
    print("=" * 70)
    print("MODEL EVALUATION REPORT")
    print("=" * 70)

    print(f"Train→Val Generalization Ratio (normalized MAE): {train_val_ratio:.2f}x")
    print(f"Val→Test Generalization Ratio (normalized MAE): {val_test_ratio:.2f}x")
    print()

    print("MAE Results:")
    print(f"Training MAE (normalized): {norm_train_mae:.4f} ({norm_train_mae*100:.2f}%)")
    print(f"Validation MAE (normalized): {norm_val_mae:.4f} ({norm_val_mae*100:.2f}%)")
    print(f"Test MAE (normalized): {norm_test_mae:.4f} ({norm_test_mae*100:.2f}%)")
    print()

    print(f"Training MAE (original): {train_mae:.2f} bu/acre")
    print(f"Validation MAE (original): {val_mae:.2f} bu/acre")
    print(f"Test MAE (original): {test_mae:.2f} bu/acre")
    print()

    print("MSE Results:")
    print(f"Training MSE (normalized): {norm_train_mse:.4f}")
    print(f"Validation MSE (normalized): {norm_val_mse:.4f}")
    print(f"Test MSE (normalized): {norm_test_mse:.4f}")
    print()

    print(f"Training MSE (original): {train_mse:.2f} (bu/acre)²")
    print(f"Validation MSE (original): {val_mse:.2f} (bu/acre)²")
    print(f"Test MSE (original): {test_mse:.2f} (bu/acre)²")
    print()

    print("RMSE Results:")
    print(f"Training RMSE (normalized): {norm_train_rmse:.4f}")
    print(f"Validation RMSE (normalized): {norm_val_rmse:.4f}")
    print(f"Test RMSE (normalized): {norm_test_rmse:.4f}")
    print()

    print(f"Training RMSE (original): {train_rmse:.2f} bu/acre")
    print(f"Validation RMSE (original): {val_rmse:.2f} bu/acre")
    print(f"Test RMSE (original): {test_rmse:.2f} bu/acre")
    print()

    # --- Performance Assessment ---
    print("=" * 70)
    print("GENERALIZATION ASSESSMENT")
    print("=" * 70)

    if val_test_ratio < 1.1:
        print("EXCELLENT generalization - Test performance matches validation!")
    elif val_test_ratio < 1.2:
        print("GOOD generalization - Minimal performance drop on test set")
    elif val_test_ratio < 1.3:
        print("ACCEPTABLE generalization - Moderate performance drop")
    elif val_test_ratio < 1.5:
        print("CONCERNING generalization - Significant performance drop")
    else:
        print(" POOR generalization - Major overfitting detected")

    print(f"\nOverall Model Performance:")
    if norm_test_mae < 0.12:  # Less than 12% error
        print(f" Excellent performance: {norm_test_mae*100:.1f}% test error rate")
    elif norm_test_mae < 0.15:  # Less than 15% error
        print(f" Good performance: {norm_test_mae*100:.1f}% test error rate")
    elif norm_test_mae < 0.20:  # Less than 20% error
        print(f"Acceptable performance: {norm_test_mae*100:.1f}% test error rate")
    else:
        print(f"Poor performance: {norm_test_mae*100:.1f}% test error rate")

    print(f"\nTarget mean (training): {train_mean:.2f} bu/acre")
    print(f"Dataset sizes - Train: {len(y_train_true)}, Val: {len(y_val_true)}, Test: {len(y_test_true)}")

    return {
        # MAE metrics
        "train_mae": train_mae,
        "val_mae": val_mae,
        "test_mae": test_mae,
        "norm_train_mae": norm_train_mae,
        "norm_val_mae": norm_val_mae,
        "norm_test_mae": norm_test_mae,

        # MSE metrics
        "train_mse": train_mse,
        "val_mse": val_mse,
        "test_mse": test_mse,
        "norm_train_mse": norm_train_mse,
        "norm_val_mse": norm_val_mse,
        "norm_test_mse": norm_test_mse,

        # RMSE metrics
        "train_rmse": train_rmse,
        "val_rmse": val_rmse,
        "test_rmse": test_rmse,
        "norm_train_rmse": norm_train_rmse,
        "norm_val_rmse": norm_val_rmse,
        "norm_test_rmse": norm_test_rmse,

        # Generalization ratios
        "train_val_ratio": train_val_ratio,
        "val_test_ratio": val_test_ratio,

        # Additional info
        "train_mean": train_mean
    }

In [35]:
results = evaluate_and_report2(
    model,
    X_static_train, X_seq_train, X_train_cols_clean, y_train,
    X_static_val, X_seq_val, X_val_cols_clean, y_val,
    X_static_test, X_seq_test, X_test_cols_clean, y_test,
  scalers['y_scaler']
)

389/389 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
MODEL EVALUATION REPORT
Train→Val Generalization Ratio (normalized MAE): 1.18x
Val→Test Generalization Ratio (normalized MAE): 0.99x

MAE Results:
Training MAE (normalized): 0.0737 (7.37%)
Validation MAE (normalized): 0.0867 (8.67%)
Test MAE (normalized): 0.0861 (8.61%)

Training MAE (original): 11.54 bu/acre
Validation MAE (original): 13.59 bu/acre
Test MAE (original): 13.49 bu/acre

MSE Results:
Training MSE (normalized): 0.0093
Validation MSE (normalized): 0.0129
Test MSE (normalized): 0.0112

Training MSE (original): 229.08 (bu/acre)²
Validation MSE (original): 315.83 (bu/acre)²
Test MSE (original): 275.46 (bu/acre)²

RMSE Results:
Training RMSE (normalized): 0.0966
Validation RMSE (normalized): 0.1134
Test RMSE (normalized): 0.1059

Training RMSE (original): 15.14 bu/acre
Validation RMSE (original): 17.77 bu/acre
Test RMSE (original): 16.60 bu/acre

GENERALIZATI